


<font size='10' color = 'E3A440'>**Mégadonnées et techniques avancées démystifiées**</font>
=======
<font color = 'E3A440'>*Nouvelles méthodes d’analyse et leur implication quant à la gestion des mégadonnées en SSH (partie 1)*</font>
=============


Cet atelier s’inscrit dans le cadre de la formation [Mégadonnées et techniques avancées démystifiées](https://www.4point0.ca/2022/08/22/formation-megadonnees-demystifiees/) (séance 6).

Les sciences humaines et sociales sont souvent confrontées à l’analyse de données non structurées, comme le texte. Après avoir préparé les données, plusieurs techniques d’analyse venant de l’apprentissage automatique peuvent être utilisées. Pendant cet atelier, les participants seront initiés au prétraitement des données textuelles et aux méthodes supervisées et non supervisées à des buts d’analyse avec Python.

Note : Cet atelier se poursuit lors d’une 2e séance le **10 novembre 2022**. Les deux séances ne peuvent pas être considerées exaustives du domaine.

Structure de l'atelier :
1. Partie 1 : Présentation en mode plénière de la [Section 1](#Section_1) (40 minutes)
2. Partie 2 : Travail individuel sur la [Section 2](#Section_2) (10 minutes)
3. Partie 3 : Travail en équipe sur la [Section 2](#Section_2) (30 minutes)
4. Partie 4 : Conclusion en mode plénière (10 minutes)

### Auteurs: 
- Bruno Agard <bruno.agard@polymtl.ca>
- Davide Pulizzotto <davide.pulizzotto@polymtl.ca>

Département de Mathématiques et de génie industriel

École Polytechnique de Montréal

# <font color = 'E3A440'>0. Préparation environnement </font>

In [ ]:
# Downloading of data from the GitHub project
!rm -rf Donnees_demystifiees_seance_6/
!git clone https://github.com/puli83/Donnees_demystifiees_seance_6

Cloning into 'Donnees_demystifiees_seance_6'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 85 (delta 34), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (85/85), done.


In [ ]:
# Import modules
import os
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

###  <font color = 'E3A440'>**0.1 Les bases de Python pour réaliser cet atelier**</font>

Nous présentons deux concepts de bases qui permettront aux personnes qui n'ont jamais écrit une ligne de code de s'orienter pendant cet atelier.

<font color = 'E3A440'>Les variables</font>

Les [variables](https://www.w3schools.com/python/python_variables.asp) sont des objets qui contiennent information dans de formats très variés. Pour créer une variable, nous utilisons la syntaxe suivante. `nom_variable = contenu_de_la_variable`. Voici quelque exemple: 

In [ ]:
variable_avec_caracteres = 'contenu de la variable avec caractères'

In [ ]:
print(variable_avec_caracteres)

contenu de la variable avec caractères


In [ ]:
variable_avec_nombre = 5
print(variable_avec_nombre)

5


In [ ]:
variable_avec_liste_de_nombres = [5, 6, 7, 8, 9]
print(variable_avec_liste_de_nombres)

[5, 6, 7, 8, 9]


In [ ]:
variable_avec_donnes_tabulaires = pd.DataFrame([[1,2,3],[4,5,6]], columns = ['Col_1','Col_2','Col_3'], index = ['Row_1','Row_2'])

In [ ]:
print(variable_avec_donnes_tabulaires)

       Col_1  Col_2  Col_3
Row_1      1      2      3
Row_2      4      5      6


<font color = 'E3A440'>Les fonctions</font>

Une [fonction](https://www.w3schools.com/python/python_functions.asp) permet d'accomplir des calculs à partir d'une ou plusieurs variables, des données, etc. Les fonctions ont généralement la syntaxe suivante:  `resultat_de_la_function = fonction(argument_1 = contenu_argument_1, argument_2 = contenu_argument_2, etc.)`. Voici quelque exemple: 

In [ ]:
resultat_somme = sum(variable_avec_liste_de_nombres)
print(resultat_somme)

35


In [ ]:
pd.DataFrame( data = [[1,2,3],[4,5,6]], columns = ['Col_1','Col_2','Col_3'], index = ['Row_1','Row_2'] )

,Col_1,Col_2,Col_3
Row_1,1,2,3
Row_2,4,5,6


<a name='Section_1'></a>
# <font color = 'E3A440'>1. *Préparation des données textuelles*</font>

L'analyse de données textuelles implique la transformation d'un texte en un objet mathématique qui peut être utilisé par des algorithmes et des modèles statistiques. Cette étape est importante car elle permet de **structurer** des données non structurées, comme le texte.


###  <font color = 'E3A440'>**1.1 Étapes fondamentales du prétraitement**</font>

Prenons la phrase suivante pour illustrer les étapes qui nous permettront de la transformer en information structurée.

In [ ]:
sentence = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good."""

Pour l'instant, `sentence` est simplement une chaine de carractères. On peut compter le nombre de caractères qui composent cette variable.

In [ ]:
len(sentence)

78

Connaitre le nombre de caractères dans un texte, n'est peut être pas suffisant pour analyser son contenu :-).

Nous allons voir dans la suite différents outils d'analyse.

#### <font color = 'E3A440'>*a. Tokenisation*</font>

Tout d'abord, il peut être utile de découper la chaine de caractères initiale en unités linguistiques élémentaires et dotées de sens, généralement appelés  "mots".

Dans le module `nltk`, il existe une fonction (`word_tokenize()`) qui permet de réaliser cette opération.

In [ ]:
# La function word_tokenize() prend la phrase comme argument.
words = nltk.word_tokenize(sentence)
print(words)
len(words)

['At', 'eight', "o'clock", ',', 'on', 'Thursday', 'morning', ',', 'the', 'great', 'Arthur', 'did', "n't", 'feel', 'VERY', 'good', '.']


17

#### <font color = 'E3A440'>*b. Analyse morphosyntaxique*</font>

Après avoir identifé tous les mots, il est possible d'analyser leur rôle morphosyntaxique, à des fins d'analyse et/ou filtrage. 

In [ ]:
# La function word_tokenize() prend la liste de mots comme argument.
words_pos = nltk.pos_tag(words, tagset='universal')
print(words_pos)
len(words_pos)

[('At', 'ADP'), ('eight', 'NUM'), ("o'clock", 'NOUN'), (',', '.'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), (',', '.'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ("n't", 'ADV'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ'), ('.', '.')]


17

Voici la liste de possibles POS tags:

| **POS** | **DESCRIPTION**           | **EXAMPLES**                                      |
| ------- | ------------------------- | ------------------------------------------------- |
| ADJ     | adjective                 | big, old, green, incomprehensible, first      |
| ADP     | adposition                | in, to, during                                |
| ADV     | adverb                    | very, tomorrow, down, where, there            |
| AUX     | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ    | conjunction               | and, or, but                                  |
| CCONJ   | coordinating conjunction  | and, or, but                                  |
| DET     | determiner                | a, an, the                                    |
| INTJ    | interjection              | psst, ouch, bravo, hello                      |
| NOUN    | noun                      | girl, cat, tree, air, beauty                  |
| NUM     | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART    | particle                  | ’s, not                                      |
| PRON    | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN   | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT   | punctuation               | ., (, ), ?                                    |
| SCONJ   | subordinating conjunction | if, while, that                               |
| SYM     | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :)               |
| VERB    | verb                      | run, runs, running, eat, ate, eating          |
| X       | other                     | sfpksdpsxmsa                                  |
| SPACE   | space                     |                                                   |


#### <font color = 'E3A440'>*c. Retirer la ponctuation*</font>

Une autre opération consiste à retirer la ponctuation. Ce type de filtrage réduit le nombre de signes graphiques qui participent le moins à la construction de la sémantique de la phrase. 
Dans certains contextes, comme en stylométrie, ce processus peut être exécuté avec des techniques plus sophistiquées. 

In [ ]:
# La ligne de code suivant itère sur chaque "mot" et retient ceux qui composés uniquement de caractères alphanumériques.
words_pos1 = [(w, pos) for w, pos in words_pos if w.isalnum()]
print(words_pos1)
len(words_pos1)

[('At', 'ADP'), ('eight', 'NUM'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ')]


12

In [ ]:
# Il serait possible aussi d'utiliser le résultat de l'analyse morphosyntaxique pour éliminer la ponctuaction
words_pos2 = [(w, pos) for w, pos in words_pos if pos != '.']
print(words_pos2)
len(words_pos2)

[('At', 'ADP'), ('eight', 'NUM'), ("o'clock", 'NOUN'), ('on', 'ADP'), ('Thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('Arthur', 'NOUN'), ('did', 'VERB'), ("n't", 'ADV'), ('feel', 'VERB'), ('VERY', 'ADV'), ('good', 'ADJ')]


14

Remarquez la différence: les "mots" [o'clock] et [n't] sont absents de la première liste, mais présents dans la seconde.

In [ ]:
words_pos = words_pos2

#### <font color = 'E3A440'>*d. Convertir chaque caractère en minuscule*</font>

Cette étape constitue une première opération de normalisation des mots et de leur réduction à une forme graphique unique. Ce genre d'étape permet de regrouper chaque occurence d'un mot sous une seule forme.

In [ ]:
# La ligne de code suivant itère sur chaque signe graphique et le transforme en minuscule.
words_pos = [(w.lower(), pos) for w, pos in words_pos]
print(words_pos)

[('at', 'ADP'), ('eight', 'NUM'), ("o'clock", 'NOUN'), ('on', 'ADP'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('the', 'DET'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('did', 'VERB'), ("n't", 'ADV'), ('feel', 'VERB'), ('very', 'ADV'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*e. Retirer les stopwords (mots vides)*</font>

Une autre opération de filtrage consiste à éliminer de mots fonctionnels. Cette liste de mots contient tout les connecteurs de phrases, comme "et", "mais", "toutefois" et des mots avec une faible valeur sémantique, comme les verbes modaux. 
Comme d'autres opération de filtrage, l'enjeux est celui de nettoyer le plus possible le vocabulaire et de reduire toutes les occurrences d'un mot sous une forme graphique unique.

In [ ]:
# Nous importons la liste de stopword en anglais
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# La ligne de code suivant itère sur chaque mot et garde ceux qui ne sont pas dans la liste de stopword.
words_pos = [(w, pos) for w, pos in words_pos if w not in stopwords.words("english")]
print(words_pos)
len(words_pos)

[('eight', 'NUM'), ("o'clock", 'NOUN'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ("n't", 'ADV'), ('feel', 'VERB'), ('good', 'ADJ')]


9

#### <font color = 'E3A440'>*f. Rammener les mots à leur racine*</font> 

En suivant le même objectif, nous retirons le suffixe morphologique des mots, ce qui augmente le niveau de réduction de chaque occurrence d'un mot à une unique forme graphique.

Il existe deux méthode fondamentales: la racinisaiton et la lemmatisation.
La première réduit les occurences à une racine qui est inférée au moyen de plusieur techniques, l'autre est la réduction de l'occurrence à son lemme. 

In [ ]:
# Racinisation: technique Porter
from nltk.stem.porter import PorterStemmer
stemmed_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ("o'clock", 'NOUN'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ("n't", 'ADV'), ('feel', 'VERB'), ('good', 'ADJ')]


In [ ]:
# Racinisation: technique Lancaster
from nltk.stem import LancasterStemmer
stemmed_pos = [(LancasterStemmer().stem(w), pos) for w, pos in words_pos]
print(stemmed_pos)

[('eight', 'NUM'), ("o'clock", 'NOUN'), ('thursday', 'NOUN'), ('morn', 'NOUN'), ('gre', 'ADJ'), ('arth', 'NOUN'), ("n't", 'ADV'), ('feel', 'VERB'), ('good', 'ADJ')]


In [ ]:
# Lemmatisaiton: utilisant le thesaurus wordnet
from nltk.stem.wordnet import WordNetLemmatizer
lemmed_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
print(lemmed_pos)

[('eight', 'NUM'), ("o'clock", 'NOUN'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ("n't", 'ADV'), ('feel', 'VERB'), ('good', 'ADJ')]


#### <font color = 'E3A440'>*g. Filtrage selon le rôle morphosyntaxique*</font>

Le filtrage des unités lexicales peut s'étendre jusqu'à l'élimination d'unités qui ne font pas partie d'une liste de rôles morphosyntaxique prédéfinie. Par exmeple, nous pouvons retirer tous les mots qui ne sont pas de *noms* ou des *adjectifs*.

In [ ]:
# Retenir seulement les noms et les adjectifs
lemmed_pos = [(w, pos) for w, pos in words_pos if pos in ['NOUN','ADJ']]
print(lemmed_pos)

[("o'clock", 'NOUN'), ('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('good', 'ADJ')]


## <font color = 'E3A440'>**1.2 Traitement d'un corpus**</font>

Le prétraitement d'un corpus de textes peut nécessiter de la mise en place de plusieurs étapes. La première et la plus importante est le découpage du corpus. 

### <font color = 'E3A440'>*1. Découpage du texte*</font>

Tout dépendant de l'objectif de l'analyse, le texte peut être découpé en plusieurs fragments, chacun desquels peut être un document, un paragraphe, une concordance, un groupe de phrases, une phrase simple, etc.



In [ ]:
text = """At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.
          The following morning, at nine, Arthur felt better.
          A dog run in the street."""
len(text)

175

Dans le bloc de code suivant, nous faisons un découpage par phrase.

In [ ]:
sentences = nltk.sent_tokenize(text)
print(sentences)
len(sentences)

["At eight o'clock, on Thursday morning, the great Arthur didn't feel VERY good.", 'The following morning, at nine, Arthur felt better.', 'A dog run in the street.']


3

### <font color = 'E3A440'>*2. Annotation et nettoyage*</font>

Les opérations précédentes d'annotation morphosyntaxique et de filtrage seront appliquées à chaque fragment du corpus qui a été créé.


#### <font color = 'E3A440'>*a. Création d'une fonction*</font>

Dans le code suivant, une fonction est créé pour englober toutes les opérations nécessaires pour l'annotation et le nettoyage.

In [ ]:
# To run this function proprlely, you need to import modules needed
def CleaningText(text_as_string, language = 'english', reduce = '', list_pos_to_keep = [], Stopwords_to_add = []):
    from nltk.corpus import stopwords

    words = nltk.word_tokenize(text_as_string)
    words_pos = nltk.pos_tag(words, tagset='universal')
    words_pos = [(w, pos) for w, pos in words_pos if w.isalnum()]
    words_pos = [(w.lower(), pos) for w, pos in words_pos]
    
    if reduce == 'stem': 
        from nltk.stem.porter import PorterStemmer
        reduced_words_pos = [(PorterStemmer().stem(w), pos) for w, pos in words_pos]
        
    elif reduce == 'lemma':
        from nltk.stem.wordnet import WordNetLemmatizer
        reduced_words_pos = [(WordNetLemmatizer().lemmatize(w), pos) for w, pos in words_pos]
    else:
        import warnings
        reduced_words_pos = words_pos
        warnings.warn("Warning : any reduction was made on words! Please, use \"reduce\" argument to chosse between 'stem' or  'lemma'")
    if list_pos_to_keep:
        reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if pos in list_pos_to_keep]
    else:
        import warnings
        warnings.warn("Warning : any POS filtering was made. Please, use \"list_pos_to_keep\" to create a list of POS tag to keep.")
    
    list_stopwords = stopwords.words(language) + Stopwords_to_add
    reduced_words_pos = [(w, pos) for w, pos in reduced_words_pos if w not in list_stopwords and len(w) > 1 ]
    return reduced_words_pos



#### <font color = 'E3A440'>*b. Application du nettoyage*</font>

Maintenant, nous pouvons apliquer cette function à chaque fragment de texte.

In [ ]:
cleaned_sentences = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['NOUN','ADJ','VERB']) for sent in sentences]
print(cleaned_sentences)

[[('thursday', 'NOUN'), ('morning', 'NOUN'), ('great', 'ADJ'), ('arthur', 'NOUN'), ('feel', 'VERB'), ('good', 'ADJ')], [('following', 'ADJ'), ('morning', 'NOUN'), ('arthur', 'NOUN'), ('felt', 'VERB')], [('dog', 'NOUN'), ('run', 'NOUN'), ('street', 'NOUN')]]


#### <font color = 'E3A440'>*c. Fréquence des mots*</font>

Quelle est la fréquence des mots de notre corpus? Pour répondre, nous créons une liste de mots en retirant l'annotation morphosyntaxique.

In [ ]:
freqs_in_text = nltk.FreqDist([w for sent in cleaned_sentences for w, pos in sent ])
freqs_in_text

FreqDist({'morning': 2, 'arthur': 2, 'thursday': 1, 'great': 1, 'feel': 1, 'good': 1, 'following': 1, 'felt': 1, 'dog': 1, 'run': 1, ...})

### <font color = 'E3A440'>*3. Vectorisation*</font>

Généralement, pour utiliser le texte dans un contexte d'analyse de données ou d'apprentissage automatique, ce texte doit être transformé dans un objet mathématique approprié. 
Le modèle le plus simple et diffusé est le "sac de mots" ("bags-of-words"), dans lequel chaque texte (ou chaque fragment de texte) est défini dans un vecteur, par un certain nombre d'unités lexicales qui le caractérisent. Ce modèle appartient à la famille de modèles de la sémantique vectorielle et il a la forme suivante:


$$X = \begin{bmatrix} 
x_{1,1} & x_{1,2} & \ldots & x_{1,w} \\
\vdots & \vdots       &  \ddots      & \vdots \\ 
x_{n,1} & x_{1,2} & \ldots & x_{n,w} \\
\end{bmatrix}
$$ 

Dans cette matrice, la valeur $x_{i,j}$ représente le "poids" du mot $j$ dans le texte $i$. Ce poids peut prendre différentes valeurs selon ce que l'on cherche à faire. Ainsi :

- $x_{i,j}$ peut représenter la présence du mot "j" dans le texte $i$,
- $x_{i,j}$ peut mesurer le nombre d'occurences du mot $j$ dans le texte $i$,
- $x_{i,j}$ peut représenter l'**importance** du mot $j$ dans le texte $i$, dans ce cas on utilisera par exemple la métrique tf-idf :
 $$\text{tf-idf}_{i,j}=\text{tf}_{i,j}.log\left(\frac{n}{n_i}\right)$$
 - $\text{tf}_{i,j}$ est la fréquence du terme $i$ dans le document $j$,
 - $n$ nombre total de documents dans l’ensemble de textes à étudier,
 - $n_i$ nombre de documents dans l’ensemble de textes qui contiennent le terme $i$.


In [ ]:
# Initialisation de l'objet
from nltk.corpus import stopwords

def identity_tokenizer(text):
    return text

# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 1, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 10, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

Utilisation du "vectorizer" avec une liste de listes de mot (et non une liste de tuple de mots-pos).

In [ ]:
# Liste de liste de mots:
[[w for w, pos in sent] for sent in cleaned_sentences]

[['thursday', 'morning', 'great', 'arthur', 'feel', 'good'],
 ['following', 'morning', 'arthur', 'felt'],
 ['dog', 'run', 'street']]

In [ ]:
# Application du vectorizer
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_sentences])
print(pd.DataFrame(freq_term_DTM.todense(), columns =  [k for k, v in sorted(vectorized.vocabulary_.items(), key=lambda item: item[1])] ))

   arthur  dog  feel  felt  following  good  great  morning  run  street  \
0       1    0     1     0          0     1      1        1    0       0   
1       1    0     0     1          1     0      0        1    0       0   
2       0    1     0     0          0     0      0        0    1       1   

   thursday  
0         1  
1         0  
2         0  


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


Thus, we assign the result of the Tf-IDF weighting to the variable named `tfidf_DTM`. 

In [ ]:
# Calculate the tfidf matrix
tfidf = TfidfTransformer(norm='l1')
tfidf_DTM = tfidf.fit_transform(freq_term_DTM)
print(pd.DataFrame(tfidf_DTM.todense(), columns =  [k for k, v in sorted(vectorized.vocabulary_.items(), key=lambda item: item[1])] ))

     arthur       dog      feel      felt  following      good     great  \
0  0.137750  0.000000  0.181125  0.000000   0.000000  0.181125  0.181125   
1  0.215994  0.000000  0.000000  0.284006   0.284006  0.000000  0.000000   
2  0.000000  0.333333  0.000000  0.000000   0.000000  0.000000  0.000000   

    morning       run    street  thursday  
0  0.137750  0.000000  0.000000  0.181125  
1  0.215994  0.000000  0.000000  0.000000  
2  0.000000  0.333333  0.333333  0.000000  


<a name="Section_2"></a>
# <font color = 'E3A440'> 2. *Exercise : Analyse de sentiment sur Twitter* </font>

L'exercice qui est proposé dans cette section est basé sur une simple chaîne de traitement pour l'**analyse de sentiments** sur des données de Twitter et sur l'**analyse de spécificités lexicales**. 

Le corpus utilisé a été collecté en 2020 par *trackmyhashtag.com* et contient 150 000 tweets pour les 50 profils les plus suivis de Twitter. Les données sont en format tabulaire dans un fichier CSV. Pour des raisons pédagogiques, cet exercice prévoit l'utilisation d'un échantillon aléatoire de 5 000 tweets.

Dans un premier temps, les données textuelles de 5 000 tweets seront analysées par un module d'analyse de sentiments du module `nltk`. Ensuite, le texte sera prétraité et certaines analyses lexicales seront executées.

Pendant l'exercice, le participant sera invité à remplir les parties manquantes du code qui sont indiquées avec `...` (trois points).

## <font color = 'E3A440'> 2.1 Présentation de l'exercice </font>

### <font color = 'E3A440'> a. Importer les données </font>

Le fichier avec les données est archivé dans un `.zip` et contient plus de 150 000 tweets. Pour de raisons pédagogiques, nous importons seulement 5 000 tweets de façon aléatoire. 

In [ ]:
ROOT_DIR='Donnees_demystifiees_seance_6/'
DATA_DIR=os.path.join(ROOT_DIR, 'Data')
import zipfile
from datetime import datetime

#Unzips the dataset and gets the TSV dataset
with zipfile.ZipFile(os.path.join(DATA_DIR,'4POINT0_Top_50_tweet_profiles.zip'), 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)

df = pd.read_pickle(os.path.join(DATA_DIR,'Top_50_tweet_profiles.pkl')).sample(5000, random_state = 5641).reset_index()

Voici les noms de variables disponibles.

In [ ]:
df.columns

Index(['index', 'Tweet Id', 'Tweet URL', 'Tweet Posted Time', 'Tweet Content',
       'Tweet Type', 'Client', 'Retweets received', 'Likes received',
       'User Id', 'Name', 'Username', 'Verified or Non-Verified',
       'Profile URL', 'Protected or Not Protected', 'Profile Account'],
      dtype='object')

Voici une observation (une ligne du tableau de données):

In [ ]:
df.iloc[0]

index                                                                     71560
Tweet Id                                                     656538552327630848
Tweet URL                     https://twitter.com/billboard/status/656538552...
Tweet Posted Time                                           2015-10-20 18:32:43
Tweet Content                 .@JustinBieber, @Skrillex and @Bloodpop's #Sor...
Tweet Type                                                              Retweet
Client                                                       Twitter Web Client
Retweets received                                                         20260
Likes received                                                            22109
User Id                                                                 9695312
Name                                                                  billboard
Username                                                              billboard
Verified or Non-Verified                

### <font color = 'E3A440'> b. Exécuter l'analyse de sentiments</font>

L'objet `SentimentIntensityAnalyzer` est utilisé pour exécuter l'analyse de sentiments. L'objet doit être initialisé et, ensuite, la fonction `polarity_scores()` peut être appliquée à une chaîne de caractères.

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Voici trois exemple d'analyse de sentiment. Le résulat de la fonction `polarity_scores()` retourne quatre valeurs: 

 1. `neg` : indique le dégré, dans une échelle de 0 à 1, de sentiment négatif du texte.
 2. `neu` : indique le dégré, dans une échelle de 0 à 1, de sentiment neutre du texte.
 3. `pos` : indique le dégré, dans une échelle de 0 à 1, de sentiment positif du texte.
 4. `compound` : contient une valeur composée des trois métriques précédentes et va de -1 à 1.



In [ ]:
sia.polarity_scores("Wow, Montreal Canadiens is the greatest hockey team in the world!")

{'neg': 0.0, 'neu': 0.52, 'pos': 0.48, 'compound': 0.8516}

In [ ]:
sia.polarity_scores("Ottawa is not bad city!")

{'neg': 0.0, 'neu': 0.56, 'pos': 0.44, 'compound': 0.484}

In [ ]:
sia.polarity_scores("No, you cannot put pineapple on a pizza! This is disgusting!")

{'neg': 0.436, 'neu': 0.564, 'pos': 0.0, 'compound': -0.7339}

Voici le tweet sur lesquels nous appliquons l'analyse de sentiment:

In [ ]:
df['Tweet Content']

0       .@JustinBieber, @Skrillex and @Bloodpop's #Sor...
1       👏 ¡@SergioRamos y @hazardeden10 se encuentran ...
2       Here's how the market may predict the next pre...
3       “Children are magical on road trips. They have...
4       .@MelissaMcCarthy told me about the moment she...
                              ...                        
4995    So saddened to hear of the tragic theatre shoo...
4996    always takes the road less traveled... @ New O...
4997    #HustleHart #MoveWithHart https://t.co/GkQHkhKhR3
4998    This is the letter the US Attorney General sen...
4999    The Week on Instagram | 276\nhttps://t.co/9kIt...
Name: Tweet Content, Length: 5000, dtype: object

Dans le prochain bloc de code, nous exécutons l'anayse de sentiment dur la colonne `Tweet Content`, et nous ajoutons  les résulats obtenus au tableau des données, l'objet nommé `df`.

In [ ]:
# Exécution de l'analyse de sentiments sur tout le corpus
datasent = df.apply(lambda x: sia.polarity_scores(x['Tweet Content']), 1)
df = df.join(pd.DataFrame(list(datasent)))

Le résultat de l'analyse est enregistré sous forme de variables. Voici un exemple:

In [ ]:
df.iloc[0]

index                                                                     71560
Tweet Id                                                     656538552327630848
Tweet URL                     https://twitter.com/billboard/status/656538552...
Tweet Posted Time                                           2015-10-20 18:32:43
Tweet Content                 .@JustinBieber, @Skrillex and @Bloodpop's #Sor...
Tweet Type                                                              Retweet
Client                                                       Twitter Web Client
Retweets received                                                         20260
Likes received                                                            22109
User Id                                                                 9695312
Name                                                                  billboard
Username                                                              billboard
Verified or Non-Verified                

Pour rendre simple l'analyse, nous utiliserons seulement la métrique composée `compound` qui est automatiquement calculée par la fonction  `polarity_score()`.

In [ ]:
df['compound'].describe()

count    5000.000000
mean        0.199087
std         0.417216
min        -0.972600
25%         0.000000
50%         0.000000
75%         0.557400
max         0.980200
Name: compound, dtype: float64

Pour utiilser la métrique `compound` dans un conteste d'**analyse de spécificité lexicale**, il est nécessaire de constituer des catégories, soit de regrouper les tweets sous les catégories suivantes: :
 1. `negative` : qui regroupe les tweets contenant un sentiment négatif (`compound` de -1 à -0.1)  
 2. `neu` : qui regroupe les tweets plustôt neutres (`compound` de -0.5 à 0.5)
 3. `positive` : qui regroupe les tweets contenant un sentiment positif (`compound` plus de 0.5)

In [ ]:
# 1 Déterminer les valeurs pour couper la métrique compound
bins = [-1, -0.1, 0.5, 1]
# 2 Déterminer les noms des categoris. NOTEZ que les nombres de noms de catégories doivent être inferieure aux valeur de découpage.
names = ['negative', 'neu', 'positive']
# Exécuter le decoupage avec la fonction 'cut' de pandas.
df['compound_category']  = pd.cut(df['compound'], bins, labels=names, include_lowest =True)

Voici la distribution des tweets par catégorie:

In [ ]:
Counter(df['compound_category'])

Counter({'negative': 630, 'neu': 2969, 'positive': 1401})

### <font color = 'E3A440'> c. Annotation, nettoyage et vectorisation des tweets </font>

Nous utilisons la fonction écrite précédemement pour nettoyer les unités lexicales de tweets. Pour ce premier test, nous conservons seulement les adjectifs.

Cette opération prendra quelques secondes. 

In [ ]:
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = ['ADJ'], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

Dans l'étape de vectorisation nous retenons les mots qui apparaissent dans au moins 5 documents (`min_df = 5`).

In [ ]:
# Initialisation de l'objet
def identity_tokenizer(text):
    return text
# Transforming the word in frequencies
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 5, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

In [ ]:
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])
freq_term_DTM

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'", 'b', 'c', 'e', 'f', 'g', 'h', 'j', 'l', 'n', 'p', 'r', 'u', 'v', 'w'] not in stop_words.
  % sorted(inconsistent)


<5000x172 sparse matrix of type '<class 'numpy.int64'>'
	with 2661 stored elements in Compressed Sparse Row format>

### <font color = 'E3A440'> d. Analyse des specificités lexicales </font>

L'analyse de spécificités lexicales permet de mettre en évidence les unités lexicales qui sont spécifiques à un groupe particulier de données. Dans notre cas, il est possible d'identifier les mots qui sont plus fortement associés avec des sentiments positif ou négatif. 

Pour se faire, nous utilisons une métrique très diffusée en lexicometrie, qui est la fonction de vraisemblance (log-likelihood Ratio). La métrique est basée sur cet [article](https://aclanthology.org/J93-1003.pdf). D'autres méthodes peuvent être utilisées, comme l'information mutuelle, le chi2 ou la ponderation tf-idf.

In [ ]:
def GetLexicalSpecificities(freq_term_DTM, logical_vector):
    # This code ref takes inspiration from this python module : https://pypi.org/project/corpus-toolkit/
    # and its main script:  https://github.com/kristopherkyle/corpus_toolkit/blob/master/corpus_toolkit/corpus_tools.py
    # which is based on this paper: https://aclanthology.org/J93-1003/
    import math
    df_freq_target = pd.DataFrame(np.asarray(freq_term_DTM[logical_vector].sum(0).T).reshape(-1))
    df_freq_target.index = [word for (word,idx) in sorted(vectorized.vocabulary_.items(), key= lambda x:x[1])]
    df_freq_target.columns = ['freq1']
    df_freq_target['freq2'] = np.asarray(freq_term_DTM[~(logical_vector)].sum(0).T).reshape(-1)
    df_freq_target['tot'] = df_freq_target['freq1'] + df_freq_target['freq2']

    df_freq_target['freq1'] = df_freq_target['freq1'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    df_freq_target['freq2'] = df_freq_target['freq2'].apply(lambda x: 0.0000001 if x == 0 else x).astype(float)
    #
    df_freq_target['freq1_norm'] = df_freq_target['freq1']/df_freq_target['freq1'].sum() * 1000000
    df_freq_target['freq2_norm'] = df_freq_target['freq2']/df_freq_target['freq2'].sum() * 1000000
    #
    df_freq_target['fraction'] = df_freq_target['freq1_norm'] / df_freq_target['freq2_norm']
    df_freq_target['Log-likelihood Ratio'] = df_freq_target['fraction'].apply(math.log2)
    frequency_threshold = 10 # Insert your frequency threshold as integer
    return df_freq_target[df_freq_target['tot'] > frequency_threshold]['Log-likelihood Ratio'].sort_values(ascending=False).iloc[range(50)]

Pour exécuter l'analyse de spécificité, il est nécessaire de créer un vecteur logique (avec des valeurs binaires) qui indique par `True` la classe pour laquelle nous voulons analyser la spécificité lexicale et par `False` le reste du corpus. 

In [ ]:
logical_vector = df['compound_category'] == 'positive'
logical_vector

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Name: compound_category, Length: 5000, dtype: bool

In [ ]:
sum(logical_vector)

630

Exécuter la fonction avec la matrice des fréquences (`freq_term_DTM`) et le vecteur logique que nous avons créé plus haut.

In [ ]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

free          27.839934
beautiful     27.772819
amazing        5.374933
happy          4.477503
great          3.618858
perfect        3.448933
best           3.086363
proud          2.586437
special        2.496239
much           1.311430
good           1.246304
whole          1.127005
incredible     0.957080
important      0.934360
nice           0.863971
excited        0.827445
better         0.711968
powerful       0.711968
favorite       0.629506
single         0.319650
huge           0.296930
hard           0.296930
funny          0.127005
big            0.127005
exclusive     -0.024998
young         -0.065640
able          -0.136029
sure          -0.153103
many          -0.168451
ready         -0.194923
top           -0.220918
old           -0.235565
high          -0.288032
last          -0.407331
long          -0.525071
available     -0.661491
american      -0.661491
black         -0.680350
new           -0.757518
next          -0.811594
true          -0.872995
le            -0

## <font color = 'E3A440'> 2.2 Exercice </font>

Pendant l'exercice, le participant sera invité à remplir les parties manquantes du code qui sont indiquées avec `...` (trois points).

Plusieurs manipulations et différents résultats seront demandés. Chaque sous-exercice suit la chaîne de traitement suivante :

1. Annotation et nettoyage des tweets : le participant devra ajuster quelques paramètres de la fonction pour choisir un filtrage spécifique.
2. Vectorisation:  le participant devra ajuster quelques parametres de la fonction pour choisir un filtrage spécifique.
3. Création d'un vecteur logique pour définir le groupe cible et le groupe de référence. 
4. Application de la fonction `GetLexicalSpecificities()` pour obtenir les 50 mots les plus spécifiques au groupe cible.




### <font color = 'E3A440'> a. Étudier l'impact du filtrage morphosyntaxique sur les spécificités lexicales </font>

Au point 2.1, seulement les adjectifs ont été étudiés. Faites maintenant une étude sur les noms, adjectifs et verbes et ensuite sur d'autres combinaisons qui vous interessent. 
Voici la liste des POS tag existants:

| **POS** | **DESCRIPTION**           | **EXAMPLES**                                      |
| ------- | ------------------------- | ------------------------------------------------- |
| ADJ     | adjective                 | big, old, green, incomprehensible, first      |
| ADP     | adposition                | in, to, during                                |
| ADV     | adverb                    | very, tomorrow, down, where, there            |
| AUX     | auxiliary                 | is, has (done), will (do), should (do)        |
| CONJ    | conjunction               | and, or, but                                  |
| CCONJ   | coordinating conjunction  | and, or, but                                  |
| DET     | determiner                | a, an, the                                    |
| INTJ    | interjection              | psst, ouch, bravo, hello                      |
| NOUN    | noun                      | girl, cat, tree, air, beauty                  |
| NUM     | numeral                   | 1, 2017, one, seventy-seven, IV, MMXIV        |
| PART    | particle                  | ’s, not                                      |
| PRON    | pronoun                   | I, you, he, she, myself, themselves, somebody |
| PROPN   | proper noun               | Mary, John, London, NATO, HBO                 |
| PUNCT   | punctuation               | ., (, ), ?                                    |
| SCONJ   | subordinating conjunction | if, while, that                               |
| SYM     | symbol                    | $, %, §, ©, +, −, ×, ÷, =, :)               |
| VERB    | verb                      | run, runs, running, eat, ate, eating          |
| X       | other                     | sfpksdpsxmsa                                  |
| SPACE   | space                     |                                                   |


Insérez la bonne valeur pour l'argument `list_pos_to_keep` afin de pouvoir garder les noms, adjectifs et verbers, ou toute autres combination de POS tag de votre intérêt

In [ ]:
# 1. Annotation and cleaning : ADD adjective and verbs as POS tag to keep
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = [...], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

Changez les paramètres `min_df` afin de ne pas dépasser **750 mots** de votre matrice <font color='E3A440'>**Document-Term matrix**</font>, qui est enregistrée dans l'objet `freq_term_DTM`.

Notez bien que dans cette fonction le paramètre `ngram_range` est configuré pour avoir les unigrams et les bigrams (sa valeur est : `(1,2)`).

In [ ]:
# 2. Vectorisation
def identity_tokenizer(text):
    return text
    
## 2.1 initialise with parameters : 
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = ..., # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 2), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

#
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])

freq_term_DTM

En utilisant le travail fait déjà au point **b.** de la section **2.1**, choisissez la catégorie de sentiment pour laquelle vous voulez étudier la spécificité lexicale ex.  `negative` or `positive`.

In [ ]:
logical_vector = df['compound_category'] == ...

En utilisant la fonction définie au point **d.** de la section **2.1**, ajoutez les arguments fondamentaux de la fonction, soit la matrice  <font color='E3A440'>**Document-Term matrix**</font> et le **vecteur logique** créé dans le bloc de code précédent.

In [ ]:
GetLexicalSpecificities(..., ...)

### <font color = 'E3A440'> b. Étudier de nouvelles categories basées sur le nombre de Retweet </font>

Sur tweeter, il est possible de retweetter un tweet existant. Le nombre de retweet peut être considéré comme un indicateur du suivi qu'un tweet a obtenu. En accomplissant les étapes apprises au long de cet atelier, quelles sont les spécificités lexicales des tweets qui ont eu un très grand suivi ? 

Voici la distribution de la colonne `Retweets received`.

In [ ]:
df['Retweets received'].describe()

En suivant les percentile qui sont affichés dans la distribution de la colonne `Retweets received` (résultat du bloc de code précedent), ajoutez la valeur de découpage manquante dans la liste des "bins". 
Nous diviserons le nombre de Retweet en quatres catégories: 
1. `low`, qui regroupe les tweets ayant récu un faible suivi
2. `medium`, qui regroupe les tweets ayant récu un suivi moyen
3. `high`, qui regroupe les tweets ayant récu un grand suivi
4. `very_high`, qui regroupe les tweets ayant récu un très grand suivi

In [ ]:
bins = [-np.inf, 161, ..., ..., 449711]
names = ['low', 'medium', 'high', 'very_high']
df['Retweets_received_category']  = pd.cut(df['Retweets received'], bins, labels=names, include_lowest =True)

Choisir la categorie cible pour laquelle analyser les spécificités lexicales. La valeur doit être une des quatres valeurs contenues dans la colonne `Retweets_received_category` générée dans le bloc de code précedent.

In [ ]:
logical_vector = df['Retweets_received_category'] == ...

Executez l'analyse de spécificités.

In [ ]:
GetLexicalSpecificities(freq_term_DTM, logical_vector)

### <font color = 'E3A440'> c. Étudier les languages spécifiques des profils Tweeter </font>

Dans le prochain exercice, sélectionnez deux ou trois profils Tweeter de votre choix et comparez les specificités lexicales en étudiant plusieurs combinaisons de POS tag. Quelles sont les grandes différences lexicales entre les profils que vous avez choisis ? 

Voici la liste complète des profils présents dans le corpus et enregistrés sous la colonne `Profile Account` et les nombres de tweets par profil.

In [ ]:
Counter(df['Profile Account'])

In [ ]:
# 1. Annotation and cleaning : ADD adjective and verbs as POS tag to keep
cleaned_tweets = [CleaningText(sent, reduce = 'lemma', list_pos_to_keep = [...], Stopwords_to_add=['http']) for sent in list(df['Tweet Content'])]

# 2. Vectorisation
def identity_tokenizer(text):
    return text
## 2.1 initialise with parameters : 
vectorized = CountVectorizer(lowercase = False, # Convert all characters to lowercase before tokenizing
                             min_df = 10, # Ignore terms that have a document frequency strictly lower than the given threshold 
                             max_df = 4500, # Ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words)
                             stop_words = stopwords.words('english'), # Remove the list of words provided
                             ngram_range = (1, 1), # Get the lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted
                             tokenizer=identity_tokenizer) # Override the string tokenization step while preserving the preprocessing and n-grams generation steps

#
freq_term_DTM = vectorized.fit_transform([[w for w, pos in sent] for sent in cleaned_tweets])

freq_term_DTM

In [ ]:
logical_vector = df['Profile Account'] == 'nasa'
GetLexicalSpecificities(freq_term_DTM, logical_vector)

## <font color = 'E3A440'> 2.3 NOTES PERSONNELLES: </font>

-----

-----